# Introduction to Landscape Metrics

Created by: Derek Robinson <br>
Last Updated: March 21, 2021

### Learning Objectives

In this Jupyter Notebook you will learn or review how to
<ul>
    <li> display .tif images in R; </li>
    <li> work with file, folder, and path names;</li>
    <li> resample raster data to a new resolution; </li>
    <li> calculate individual landscape metrics; </li>
    <li> merge dataframes; </li>
    <li> create a correlation matrix;</li>
    <li> conduct a variable reduction process by removing correlated variables; and </li>
    <li> report landscape metric results. </li>    
</ul>

### Assignment Completion

For those in GEOG 318/PLAN 353 @UWaterloo, the assignment directions and submission requirements are provided on LEARN as a quiz that spans multiple days. To complete the quiz/assignment you will need to do the following:

<ul>
    <li> Complete this Jupyter Notebook.</li>
    <ul>
        <li> To advance to the next text cell or execute the code in a cell hold <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift</mark></font> and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Enter</mark></font> </li>
        <li> If there is a method for which you would like to know more about its parameters or how it works then you can place your cursor on the method and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift + Tab</mark></font> and a help box will appear that you can expand and scroll through.</li>
    </ul>
    <li> Complete the Introduction to Landscape Metrics Quiz/Assignment in LEARN, which may require you to </li>
    <ul>
        <li> copy results from your notebook into the LEARN input boxes, </li>
        <li> take screen captures of your notebook output and submit them via LEARN, or </li>
        <li> change values in the Jupyter Notebook and re-execute cells/code to obtain new results. </li>
    </ul>
    <li> <b>While you may discuss course content with your classmates, you are to complete the assignment individually.</b></li>
</ul>

### Problem Statement & Data

In this Notebook we are going to use landscape metrics to quantify the composition and configuration of vegetation communities in and around multiple wetlands residing in the Rocky Mountains near Canmore, Alberta.

The following two publications provide examples of the use of landscape metrics to describe land-cover patterns. Within these papers we generate a random sample of 1-km<sup>2</sup> landscapes and then analyze the spatial pattern (using landscape metrics) across different natural areas and classes of increasing human disturbance.

Ridge, J., Robinson, D.T., and R.C. Rooney (2020). What is the distribution and structure of permanent open-water wet-landscapes in Alberta? A case for reclamation design. Wetlands Ecology and Management. (Open Access)

https://link.springer.com/article/10.1007/s11273-020-09769-2

Evans, I., Robinson, D.T., and R. Rooney (2017). A methodology for relating wetland configuration to human disturbance in Alberta. Landscape Ecology, 32(10), 2059-2076. (Open Access)

https://link.springer.com/article/10.1007/s10980-017-0566-z

**While the methods below use wetland vegetation community data, they are applicable to raster data representing other phenomena using nominal classification.** 

### Lets get started
Sometimes when running functions and code in R you will receive warnings. These are displayed in pink in Jupyter Notebook following the execution of a cell. The warnings arise for a variety of reasons like a parameter was not specified so a default will be used. In creating these notebooks I have reviewed the warnings and they do not affect the results. The next line of code turns them off so that you are not confused or worried about them. However, if you would like to see them you may restart the notebook and comment the following cell code or skip executing it.

In [ ]:
#options(warn=-1)
# command to turn warnings on
options(warn=0)

We need to use several functions in the `magick`, `landscapemetrics`, `landscapetools`, and `raster` packages. In addition to the spatial analysis methods from those packages we will use the `ggplot2` package for visualizing some of our output data. First try loading the packages and if you get an error then uncomment the install command and execute the cell to install the package. Remember, if you're working on the JupyterHub then you can't install the packages their, only on your home computer.

In [ ]:
#install.packages("landscapemetrics")
#install.packages("landscapetools")
#install.packages("raster")
#install.packages("magick")
#install.packages("rgdal")

library(magick)
library(landscapemetrics)
library(landscapetools)
library(raster)
library(rgdal)

# Displaying imagery with Magick

Lets first look at the study site with imagery acquired from a DJI Inspire 1. The two images are large ~54 and ~45 Mb each and comprise the northern and southern portions of our study wetland. The images are large because the resolution of the imagery is very-high at ~2.5 cm. To collect the imagery, the Inspire 1 was flown at a height of 60 m taking # images that were used to generate an orthomosaic in Pix4D. 

First we will load the two images into a variable `north` and `south`.

In [ ]:
north <- image_read("Wetland2BaseMap1of2.tif")
south <- image_read("Wetland2BaseMap2of2.tif")

The `magick` library can append images together but first requires a list of image objects, which we can create by concatenating the objects together using the `c()` function. Then we can specify the scale at which we want to image to display and if we want the images side-by-side then we can set the `stack` parameter to TRUE.

In [ ]:
img <- c(north, south)
image_append(image_scale(img, "500"), stack=TRUE)

# Working with multiple rasters

While the above content is useful for getting information about a single landscape. We often want to compute and summarize or compare results across multiple landscapes. First lets get a list of the wetland rasters we have available to analyze by using the `list()` function. We will get the names including the path from our Notebook location by setting the `full.names` parameter `= TRUE` and then we will get just the file names by setting that same parameter to `FALSE`. Lastly, we will use the built in R function `file_path_sans_ext()` to retrieve the file name with out the extension.

In [ ]:
wetlandFListwPath <- list.files(path='./ClassifiedRasterData/', full.names=TRUE)
wetlandFListwoPath <- list.files(path='./ClassifiedRasterData/', full.names=FALSE)

wetlandFListNames <- ""
for(i in 1:length(wetlandFListwoPath)) {
    wetlandFListNames[i] <- tools::file_path_sans_ext(wetlandFListwoPath[i])
}

Using our new list of wetland rasters, we can load our wetland rasters into raster layer objects using the `raster()` function from the `raster` package. Unfortunately we can not use a RasterStack or RasterBrick because our spatial extents and locations differ. So we have to create individual raster objects.

In [ ]:
wetlandFListwPath

Ideally, now that we have the list of file names above from our folder we could loop through those file using something like the following code


<code> 
    
wetLists = vector("list", length(wetlandFListwoPath))
wetlists <- ""
for(i in 1:length(wetlandFListwoPath)) {
    wetLists[i] <- raster(wetlandFListwPath[i])
}
</code>


However, once the rasterLayer objects are in the list I can not find a way to plot the raster using something like `plot(wetLists[1])` nor can I extract the rasterLayer for use in other functions. Therefore, we are stuck having to manually create a rasterLayer object for each `.tif`. 

In [ ]:
wetland2r <- raster("./ClassifiedRasterData//Wetland2Raster.tif")
wetland6r <- raster("./ClassifiedRasterData//Wetland6Raster.tif")
wetland7r <- raster("./ClassifiedRasterData//Wetland7Raster.tif")
wetland8r <- raster("./ClassifiedRasterData//Wetland8Raster.tif")
wetland9r <- raster("./ClassifiedRasterData//Wetland9Raster.tif")
wetland10r <- raster("./ClassifiedRasterData//Wetland10Raster.tif")
wetland11r <- raster("./ClassifiedRasterData//Wetland11Raster.tif")
wetland12r <- raster("./ClassifiedRasterData//Wetland12Raster.tif")

# Resampling Raster data

As mentioned above our raster data have different locations and spatial extents. They also have different resolutions and more than one of them has over a million pixels. We want to conduct our landscape metric analysis on data that is of the same resolution so that we can compare the landscapes with each other. There are a variety of packages and functions for resampling the data. For example, 
* the `resample()` function from the `raster` package allows you to resample and change the resolution, but it only works when the two rasters used intersect with each other. Therefore, it is a good approach if you want to snap the rasters together to share the same cell location and size. 
* the `aggregate()` function from `raster` package can also be used, but it only works by grouping cells together to make larger ones (i.e., aggregating) by a factor of the existing cell resolution.

However, the best approach is to use the `gdalUtils` package that enables us to use `gdal` in R. Using gdal not only requires that we load the package `library(gdalUtils)` but it also requires that you have gdal installed on your computer. We are lucky as it has been installed on the Math JuptyerLab so we can easily take advantage of it. Lets load the `gdalUtils` package now.

In [ ]:
library(gdalUtils)

My understanding, which could be wrong, is that because `gdalUtils` provides a wrapper for the gdal code, the functions within the package work directly with data on your computer/server rather than with R objects. For example, lets try calling the `gdalinfo()` function to ensure `gdalUtils` is parsing out the important information from our raster data. We can use `wetland2r`.

In [ ]:
gdalinfo(wetland2r)

The output from the above code has a warning message a reporting `... had status 1` error, which means that the function exited abruptly, and returned nothing useful for us. Lets try this again now, but use a direct path to the raster data stored in `ClassifiedRasterData/Wetland2Raster.tif`. You will only see the above warning if you haven't turned the warnings off at the beginning of the Notebook.

In [ ]:
gdalinfo("./ClassifiedRasterData//Wetland2Raster.tif")

___________________________________________________

You can see that the `gdalUtils` has parsed out the spatial reference information as well as the pixel size, among other information.
We are going to use the `gdalwarp()` function to resample our raster data to a common cell resolution. You can review the `gdalUtils` package and `gdalwarp()` function and parameter requirements here https://cran.r-project.org/web/packages/gdalUtils/gdalUtils.pdf 

One of the parameter requirements is an output filename. As mentioned above, because the gdalUtils is merely a wrapper it works directly with the data, so we can not port the output of `gdalwarp()` to a raster object or any other R object. 

Lets set our resolution `res` equal to 1 m. Then, because gdal only deals with files on the hard drive, we can loop through all the files in our `wetlandFListNames[]` vector and resample each raster using the `gdalwarp()` function. The first argument of the function is a file name, which we identify with the `wetlandFListwPath[i]` variable defined at the beginning of the Notebook. The second argument is a destination path and filename `dst`, which we create by adding a suffix Out.tif to our input filename using the `paste()` function. Then we add our new filename to our output path name "./ResampledData/". The third parameter `output_Raster` is asking if we want to output a raster object, which we do `TRUE`. Our forth paramter `overwrite` determines if we are going to write over an existing file with the same name, which we want to (`TRUE`) so that we can rerun the script without error. The fifth parameter `tr` defines the x and y resolution for the new cells, which we set to our `res` variable. Lastly, our sixth parameter `r` defines the type of interpolation to use, which because we are using nominal data requires that we use the nearest neighbour value that is defined as `near`.  

In [ ]:
res = 1.00

for(i in 1:length(wetlandFListNames)) {
    #output file name
    name = paste(wetlandFListNames[i],"Out.tif", sep="")
    
    # output folder plus file name
    dst = paste("./ResampledData/",name, sep="")
    
    gdalwarp(srcfile=wetlandFListwPath[i], dstfile=dst, output_Raster=TRUE, overwrite=TRUE, tr=c(res,res), r="near")    
}

Now that we have resampled all of our data to a new spatial resolution that we want to work with, we need to reload those data into raster objects for use in subsequent analysis, using the `raster()` function from the `raster` package. Even though we cannot work with the rasterLayer objects if we store them in a list, we can still use the list for other tasks as you'll soon see. So we will also add the rasters to a list `wetlandResampList` using the `as.list()` function.

In [ ]:
wetland2resamp <- raster("./ResampledData//Wetland2RasterOut.tif")
wetland6resamp <- raster("./ResampledData//Wetland6RasterOut.tif")
wetland7resamp <- raster("./ResampledData//Wetland7RasterOut.tif")
wetland8resamp <- raster("./ResampledData//Wetland8RasterOut.tif")
wetland9resamp <- raster("./ResampledData//Wetland9RasterOut.tif")
wetland10resamp <- raster("./ResampledData//Wetland10RasterOut.tif")
wetland11resamp <- raster("./ResampledData//Wetland11RasterOut.tif")
wetland12resamp <- raster("./ResampledData//Wetland12RasterOut.tif")
wetlandResampList <- as.list(wetland2resamp, wetland6resamp, wetland7resamp, wetland8resamp, wetland9resamp, wetland10resamp, wetland11resamp, wetland12resamp)

Lets view the original raster and our resampled raster side by side to visualize the outcome of our resampling. To do this we can set the plotting parameters using the `par()` function and specify a matrix format of rows `mfrow` as 1 row and two columns using `c(1,2)`. Then calling two `plot()` functions will put them in the matrix locations of 1,1 and 1,2.

In [ ]:
par(mfrow=c(1,2))
plot(wetland2r)
plot(wetland2resamp)

# Data verification for landscape analysis

The original remotely piloted aircraft (RPAS imagery was manually digitized into 12 classes of vegetation for landscape analysis. Lets confirm that the data are working with the `landscapemetrics` and `landscapetools` packages for our analysis. First, we will test that the `show_landscape()` function from the `landscapetools` package is able to plot our data.

In [ ]:
show_landscape(wetland2r)

To calculate meaningful landscape metric values with the `landscapemetrics` library requires the following three conditions to be met:

*The distance units of your projection are meter, as the package converts units internally and returns results in either meters, square meters or hectares. For more information see the help file of each function.

*Your raster encodes landscape classes as integers (1, 2, 3, 4, …, n).

*Landscape metrics describe categorical landscapes, that means that your landscape needs to be classified (we throw a warning if you have more than 30 classes to make sure you work with a classified landscape).

We can check these conditions by running the `check_landscape()` function from the `landscapemetrics` library.

In [ ]:
check_landscape(wetlandResampList)

If the data are acceptable then there will be a &#10004; in the right-most column to indicate that the data are acceptable, i.e., `OK`. You should be seeing that a coordinate reference system (crs) exists, that the units are in metres `m`,  that the classes comprise nominal data (are not doubles or floats), and the number of classes in the data.

Working with the `landscapemetrics` package to calculate landscape metrics uses a common format of `lsm_#_?`, where the `#` is replaced by a `l`, `c`, or `p`, if the analysis is at the landscape, class, or patch level, respectively. The `?` is replaced by the metric of interest, e.g., Eucliean nearest neighbour = `enn`, total area = `ta`, and many others metrics. The full list of metrics can be found here: https://r-spatialecology.github.io/landscapemetrics/reference/index.html

To see what the output looks like, lets calculate the Euclidean nearest-neighbour distance for each patch in the data layer using `lsm_p_enn()` function. We can next this function within the `head()` function to show only the first six results.

In [ ]:
head(lsm_p_enn(wetland2resamp))

The above table shows that we may have more than one layer, although we only have one. It shows what level the metric is calculated at (i.e., patch, class, or landscape), the `class` ID = 1, the patch `id` that shows each patch has its own id, the metric computed `enn`, and the `value` of that metric for each patch.

If we calculate a metric for the entire landscape, instead of for each patch, then we will only have a single row of results returned. For example, lets calculate the total area `ta` of our study area data.

In [ ]:
lsm_l_ta(wetland2resamp)

When I first observed the above table and the total area was 3.1386... I thought something was wrong. However, the data passed the `check_lanscape()`. The data are in metres `m` and the area is certainly much more than 3.146 m<sup>2</sup>... so I went to the documentation here:

https://r-spatialecology.github.io/landscapemetrics/reference/index.htmlhttps://r-spatialecology.github.io/landscapemetrics/reference/index.html

clicked on `lsm_l_ta()`, which took me here: https://r-spatialecology.github.io/landscapemetrics/reference/lsm_l_ta.html

and I found out that the output of `lsm_l_ta()` is in `hectares`, which made sense since 1 ha = 10,000 m<sup>2</sup>. 


# Landscape metric correlation and selection

Now that we have visualized and loaded raster data as well as used those data with the `landscapemetrics` package, lets work with data from multiple wetlands/rasters. Instead of picking a few metrics to use, we can calculate all the metrics at a particular level (i.e., patch, class, landscape) or any combination using the `calculate_lsm()` function. 

We do not need to calculate metrics for individual patches since we are not interested in the size or shape of every patch, but instead are interested in class and landscape metrics. However, to keep things simple, lets focus just on the landscape-level metrics.

Next lets calculate the landscape-level metrics for each wetland. **Be patient, this may take a couple of minutes.** 

In [ ]:
wetland2metrics <- calculate_lsm(wetlandResampList[1], level=c("landscape"), classes_max = 12)
wetland6metrics <- calculate_lsm(wetlandResampList[2], level=c("landscape"), classes_max = 12)
wetland7metrics <- calculate_lsm(wetlandResampList[3], level=c("landscape"), classes_max = 12)
wetland8metrics <- calculate_lsm(wetlandResampList[4], level=c("landscape"), classes_max = 12)
wetland9metrics <- calculate_lsm(wetlandResampList[5], level=c("landscape"), classes_max = 12)
wetland10metrics <- calculate_lsm(wetlandResampList[6], level=c("landscape"), classes_max = 12)
wetland11metrics <- calculate_lsm(wetlandResampList[7], level=c("landscape"), classes_max = 12)
wetland12metrics <- calculate_lsm(wetlandResampList[8], level=c("landscape"), classes_max = 12)

You can view the calculated metrics and the format of the output if you would like, but keep it short by using the `head()` function

In [ ]:
head(wetland2metrics)

Now that we have hundreds of landscape metrics for multiple wetlands we face a critical question
* **how do we report results for hundreds of metrics across multiple wetlands/landscapes?**

The simple answer is we can't since there is too much data. Instead we should be asking
* **what metrics should we report?**

An objective way to determine which metrics to report is to remove those that are highly correlated with other metrics. This process, which we have been calling `metric reduction`, is an objective way to identify landscape metrics to use in quantifying landscape composition and configuration.

To conduct a correlation analysis of our landscape metrics we first need a sequence of metric values (i.e., we need the different metric values for the same metric across all of our wetlands/landscape data). Then we can see if the values for one metric (e.g., ai) move in the same direction with the same magnitude as another metric (e.g., lpi). To do this requires that we have all the metric values from all wetlands in the same dataframe.

To get our metric values in the same data frame we first need to rename the `value` column with a name that coincides with the wetland id. We can do this using the `names()` function to assign a new name to a particular row. Since the `value` column is the sixth column we assign our new name to the sixth column name of our dataframe. Lets do that for each set of landscape metrics we just calculated.

In [ ]:
names(wetland2metrics)[6] <- "wet2value" 
names(wetland6metrics)[6] <- "wet6value" 
names(wetland7metrics)[6] <- "wet7value" 
names(wetland8metrics)[6] <- "wet8value" 
names(wetland9metrics)[6] <- "wet9value" 
names(wetland10metrics)[6] <- "wet10value" 
names(wetland11metrics)[6] <- "wet11value" 
names(wetland12metrics)[6] <- "wet12value" 

Verify the name change

In [ ]:
head(wetland2metrics)

Now you can see we change the column name from `value` to `wet2value`.

With our unique column names we can merge the landscape metric data we calcuated for each wetland together. Unfortunately I have only been able to find a way to do this using two dataframes at a time with the `merge()` function. The first two parameters are the dataframes you wish to merge and the third parameter is the column(s) you wish to merge by. If we were including class level metrics then we could alter the parameter to `by=c("class","metric")` to ensure that the correct metrics merge with the correct class (since we have 12 classes we would have 12 versions of each class-level landscape metric, one for each class).

I have built the names and the code out to be as explicit and transparent as possible here.

In [ ]:
wetlands2_6metrics <- merge(wetland2metrics, wetland6metrics, by=c("metric"))
wetlands2_6_7metrics <- merge(wetlands2_6metrics, wetland7metrics, by=c("metric")) 
wetlands2_6_7_8metrics <- merge(wetlands2_6_7metrics, wetland8metrics, by=c("metric"))
wetlands2_6_7_8_9metrics <- merge(wetlands2_6_7_8metrics, wetland9metrics, by=c("metric"))
wetlands2_6_7_8_9_10metrics <- merge(wetlands2_6_7_8_9metrics, wetland10metrics, by=c("metric"))
wetlands2_6_7_8_9_10_11metrics <- merge(wetlands2_6_7_8_9_10metrics, wetland11metrics, by=c("metric"))
wetlands2_6_7_8_9_10_11_12metrics <- merge(wetlands2_6_7_8_9_10_11metrics, wetland12metrics, by=c("metric"))

I have not yet completely cleaned the data so you may see warnings output from the previous code block noting some duplicate names and other issues. Should you feel like ensuring this is all cleaned up then please feel free to send along your improvements.

Think about what you would expect the merged data frame to look like and only after you have done this then take a quick peak at the merged data

In [ ]:
head(wetlands2_6_7_8_9_10_11_12metrics)

To clean the data.frame up, lets identify the variables we want to keep, select them out of the dataframe and put them in a new data frame.

In [ ]:
selectVars <- c("metric", "wet2value", "wet6value", "wet7value", "wet8value", "wet9value", "wet10value", "wet11value", "wet12value")
wetlandmetrics <- wetlands2_6_7_8_9_10_11_12metrics[selectVars]

I always like to view my results to ensure the steps are working out and to fully understand my data. 
Lets take a peak...

In [ ]:
wetlandmetrics

Sometimes you should take a look at all of your data, which is why I didn't use the `head()` function above. Good thing I didn't, because if you scroll through the data, you will see that some of the data are missing for a few landscape metrics and some have `NA` values. The lack of these data, especially if there is only one number in a row, can cause a problem when trying to compute the correlation between two landscape metrics. We can easily eliminate these data, by using the `complete.cases()` function which retains only those rows from the original data have data in every column.

In [ ]:
wetlandMetricsClean <- wetlandmetrics[complete.cases(wetlandmetrics), ]

Now if we view the data we should not find any missing or NA values.

In [ ]:
wetlandMetricsClean

While the dataframe above is clean and combines our landscape metrics from multiple wetlands, it is not in the format required for calculating a correlation matrix.

To calculate a correlation matrix we will use the `cor()` function from the `caret` package. Load the `caret` package.

In [ ]:
library(caret)

The `cor()` function requires the values to be correlated against each other to be in rows. To do this we can transpose our data.frame using the transpose `t()` function. However, this creates a matrix instead of a dataframe. To get the data back into a data.frame we can coerce the data using the `as.data.frame()` function. 

In [ ]:
wetMets <- as.data.frame(t(wetlandMetricsClean))

If you print out the `wetMets` object you'll see that things are working out. However, the `cor()` function will not work with the metric names occupying the first row. Lets assign the column names of the dataframe our metric names listed in row 1 using the `names()` function. Lets also delete the row from the dataframe so that the dataframe only contains numeric variables and take a look at the dataframe.

In [ ]:
names(wetMets) <- wetMets[1,]
wetMets <- wetMets[-c(1),]
wetMets

The dataframe looks good. There is only one remaining issue, which is that each column is a character `chr` data type. We can not calculate correlations between characters. To change those columns to a numeric data type, we can use the `sapply()` function to apply this conversion using the `as.numeric()` function to all columns in our dataframe.

In [ ]:
wetMets <- sapply(wetMets, as.numeric )

In [ ]:
wetMets

Finally, we can cacluate a correlation matrix to determine the degree of correlation between each landscape metric and every other landscape metric using the `cor()` function.

In [ ]:
corMatrix <- cor(wetMets)

In [ ]:
head(corMatrix)

The `findCorrelation()` function searches through a correlation matrix and identifies when two variables are correlated above a threshold value, which is specified as the `cutoff` parameter. When two variables are correlated above the cutoff value, the column index is added to a vector. You can find more details about the process of how the function determines which variable to remove and the `exact` parameter in the RDocumentation here:

https://www.rdocumentation.org/packages/caret/versions/6.0-86/topics/findCorrelation

To select a set of non-correlated landscape metrics for describing our wetland vegetation patterns, lets use the `findCorrelation()` function with a cutoff value of 0.9.

In [ ]:
metricColsToRemove <- findCorrelation(corMatrix, cutoff = 0.95, exact=TRUE)

Lets remove those columns/metrics from our matrix of landscape metrics `wetMets` using the minus sign `-` and bracket `[ , ]` notation to remove those column ids from our `wetMets` data.

In [ ]:
wetMets <- wetMets[,-metricColsToRemove]

In [ ]:
print(wetMets)

Phew... all of that work and we still have a large number of metrics left. There are still many improvements we could make to our approach and further reduce our metric selection. Typically the final metric selection is at least partially dependent on the type of research question we are answering. For examples of additional metric selection methods and research questions that are answered in part by the use of landscape metrics, review the papers listed at the beginning of this notebook. 

For now, lets simply take a subset of the non-correlated metrics we have acquired.

In [ ]:
subset <- c("shei", "sidi", "lpi", "np")
finalMetrics <- as.data.frame(t(wetlandMetricsClean))
names(finalMetrics) <- finalMetrics[1,]
finalMetrics <- finalMetrics[-c(1),]
finalMetrics <- finalMetrics[,subset]

In [ ]:
finalMetrics

Lets use some simple visualizations to inspect our four non-correlated landscape metrics and try and interpret the differences in patterns among our study wetlands.

First lets look at wetland vegetation diversity using Shannon's Evenness Index `shei` and Simpson's diversity index `sidi`. The `shei` is a measure of class dominance and equals 0 when only one patch present and equals 1 when the proportion of classes is completely equally distributed. The `sidi` is less sensitive to rare cases than Shannon's Diversity Index `shdi` and is interpreted as the probability that two randomly selected cells belong to the same class. The `sidi`= 0 when only one patch is present and approaches 1 when the number of class types increases while the proportions are equally distributed. These descriptions can be found on the Reference page of the `landscapemetrics` website https://r-spatialecology.github.io/landscapemetrics/reference/index.html


In [ ]:
library(ggplot2)
theme_set(theme_classic())

In [ ]:
g_shei <- ggplot(finalMetrics, aes(rownames(finalMetrics), shei))
g_shei + geom_bar(stat="identity", width = 0.5, fill="darkblue") + 
      labs(title="Shannon's Diverity Index") +
      theme(axis.text.x = element_text(angle=65, vjust=0.6, size = 15))

In [ ]:
g_sidi <- ggplot(finalMetrics, aes(rownames(finalMetrics), sidi))
g_sidi + geom_bar(stat="identity", width = 0.5, fill="darkblue") + 
      labs(title="Simpson's Diverity Index") +
      theme(axis.text.x = element_text(angle=65, vjust=0.6, size = 15))

In [ ]:
g_lpi <- ggplot(finalMetrics, aes(rownames(finalMetrics), lpi))
g_lpi + geom_bar(stat="identity", width = 0.5, fill="darkblue") + 
      labs(title="Largest Patch Index") +
      theme(axis.text.x = element_text(angle=65, vjust=0.6, size = 15))

In [ ]:
g_np <- ggplot(finalMetrics, aes(rownames(finalMetrics), np))
g_np + geom_bar(stat="identity", width = 0.5, fill="darkblue") + 
      labs(title="Number of Patches") +
      theme(axis.text.x = element_text(angle=65, vjust=0.6, size = 15))

The bar charts you just created only scratch the surface of many ways to analyze and visualize landscape metric values. While the bulk of this notebook was spent on working with raster data, calculating landscape metric values, and identifying non-correlated metrics for quantifying the composition and configuration of wetland vegetation patches, this approach can be used for any type of spatial pattern analysis comprising nominal (i.e., categorical) data. 

# Congratulations!

**You have reached the end of the Introduction to Landscape Metrics notebook**